In [1]:
using Distances, Statistics
using MultivariateStats
using PyPlot
using WordTokenizers
using TextAnalysis
using DelimitedFiles

In [2]:
function load_file(embedding_file)
    local LL, indexed_words, index
    indexed_words = Vector{String}()
    LL = Vector{Vector{Float32}}()
    open(embedding_file) do f
        index = 1
        for line in eachline(f)
            xs = split(line)
            word = xs[1]
            push!(indexed_words, word)
            push!(LL, parse.(Float32, xs[2:end]))
            index += 1
        end
    end
    return reduce(hcat, LL), indexed_words
end

load_file (generic function with 1 method)

In [4]:
wrd_embed, libb = load_file("C:/Users/Chetan/Desktop/glove.6B.50d.txt")
vec_size, libb_size = size(wrd_embed)
println("Downloading the word vocabulary adn peseenting as  $vec_size features. and its size is = $libb_size")

In [6]:
vec_idx(s) = findfirst(x -> x==s, libb)
vec_idx("chocolate")

6243

In [8]:
function vec(x) 
    if vec_idx(x)!=nothing
        wrd_embed[:, vec_idx(x)]
    end    
end
vec("chocolate")

50-element Array{Float32,1}:
  0.089859
  0.5691
 -0.91323
  0.34064
  0.7763
  1.3755
 -0.6681
 -0.322
 -0.061527
  0.81761
  0.1773
 -0.24408
  1.1812
  ⋮
  1.5194
 -0.54659
  0.93013
  0.17938
 -0.17086
 -0.42733
  0.75439
  1.4537
 -0.098187
 -0.59428
 -0.19965
 -0.49592

In [10]:
cos_rever_dis(x,y)=1-cosine_dist(x, y)

cos_rever_dis (generic function with 1 method)

In [11]:
cos_rever_dis(vec("wine"), vec("grapes")) > cosine(vec("chocolate"),vec("octopus"))

true

In [14]:
function near(q, k=5)
    list=[(z,cos_rever_dis(wrd_embed'[z,:], q)) for z in 1:size(wrd_embed)[2]]
    topn_idx=sort(list, by = z -> z[2], rev=true)[1:k]
    return [libb[a] for (a,_) in topn_idx]
end

near (generic function with 2 methods)

In [15]:
near(vec("chocolate")) #for 5 similar words to the input

5-element Array{String,1}:
 "chocolate"
 "cream"
 "cake"
 "caramel"
 "butter"

In [16]:
near(vec("tree"))

5-element Array{String,1}:
 "tree"
 "trees"
 "pine"
 "flower"
 "oak"

In [19]:
week_cck = vec("week") - vec("weekend")
near(week_cck + vec("monday"))

5-element Array{String,1}:
 "tuesday"
 "thursday"
 "monday"
 "wednesday"
 "friday"

In [20]:
queen_check = vec("king") - vec("man")
near(queen_check + vec("woman"))#correct answer is predicted to closest apporoximation ie. 2nd word of the array list creaeted

5-element Array{String,1}:
 "king"
 "queen"
 "daughter"
 "prince"
 "throne"